**简介**

Single Shot MultiBox Detector (SSD) 是一种单阶段的目标检测器。与两阶段的检测方法不同，单阶段目标检测并不进行区域推荐，而是直接从特征图回归出目标的边界框和分类概率。SSD 运用了这种单阶段检测的思想，并且对其进行改进：在不同尺度的特征图上检测对应尺度的目标。如下图所示，SSD 在六个尺度的特征图上进行了不同层级的预测。每个层级由两个3x3卷积分别对目标类别和边界框偏移进行回归。因此对于每个类别，SSD 的六个层级一共会产生 38x38x4 + 19x19x6 + 10x10x6 + 5x5x6 + 3x3x4 + 1x1x4 = 8732 个检测结果。



SSD 可以方便地插入到任何一种标准卷积网络中，比如 VGG、ResNet 或者 MobileNet，这些网络被称作检测器的基网络。在这个示例中我们使用 shufflenet。

在训练时还会对图片进行数据增强，包括随机扰动、扩张、翻转和裁剪:

* 扰动: 扰动图片亮度、对比度、饱和度和色相。
* 扩张: 将原始图片放进一张使用像素均值填充(随后会在减均值操作中减掉)的扩张图中，再对此图进行裁剪、缩放和翻转。
* 翻转: 水平翻转。
* 裁剪: 根据缩放比例、长宽比例两个参数生成若干候选框，再依据这些候选框和标注框的面积交并比(IoU)挑选出符合要求的裁剪结果。

也可以采用一些其他数据增广方法：

**InstaBoost:**

论文： https://arxiv.org/abs/1908.07801

代码： https://github.com/GothicAi/InstaBoost


**RandAugment：**

https://arxiv.org/pdf/1909.13719.pdf

https://github.com/tensorflow/tpu/blob/master/models/official/efficientnet/autoaugment.py

**目录**
```
| work
   |-- astar2019
      |-- score.py
      |-- ...
   |-- ssd
      |-- train.py
      |-- mobilenet_ssd.py
      |-- ...
   |-- coco
      |-- train2017
      |-- val2017
      |-- test2017
      |-- ...

```





**安装相关依赖库**

In [ ]:
python trian/train.py --learning_rate 0.008 --data_dir F:\imagesource\coco\coco2017 --epoc_num 1 --batch_size 32 --model_save_dir model/model_snet --pretrained_model model\pretrained\ShuffleNetV2_pretrained --use_multiprocess False --enable_ce True

python astar2019-evauate/score.py --batch_size 32 --data_dir F:\imagesource\coco\coco2017 --model_dir model/model_snet/300_300_inference/  
    
python trian/main_quant.py  --batch_size 16 --data_dir F:\imagesource\coco\coco2017 --epoc_num 1 --init_model model/model_snet/best_model --model_save_dir model/model_snet/snet_int8 --mode train
   
python astar2019-evauate/score.py --batch_size 16 --data_dir F:\imagesource\coco\coco2017 --model_dir model/model_snet/snet_int8/300_300_int8_inference/

Paddle已发布了图像分类模型库

https://github.com/PaddlePaddle/models/tree/develop/PaddleCV/image_classification

用户可以参照work/ssd/中的mobilenet_ssd、shufflenetv2_ssd.py、mobilenetv2_ssd.py等添加自己的backbone

**开始训练**

训练策略采用了warmup，优化器采用的Momentum，用户也可以自己更改优化器
保存的预测模型会以图片的height_width_xxx来命名

In [ ]:
#定义路径   
!python trian/train.py --learning_rate 0.008 --data_dir F:\imagesource\coco\coco2017 --epoc_num 1 --batch_size 16 --model_save_dir model/model_snet --pretrained_model model\pretrained\ShuffleNetV2_pretrained

执行score
score模型的名字需要是height_width_xxx，默认300x300

--model_dir 预测模型地址

In [9]:
!python astar2019-evauate/score.py --batch_size 8 --data_dir F:\imagesource\coco\coco2017 --model_dir model/model_snet/300_300_inference/

-----------  Configuration Arguments -----------
ap_version: cocoMAP
batch_size: 16
data_dir: dataset/coco2017
mean_value_B: 127.5
mean_value_G: 127.5
mean_value_R: 127.5
model_dir: model/model_snet/300_300_inference/
nms_threshold: 0.45
test_list: 
------------------------------------------------
loading annotations into memory...
Done (t=0.60s)
creating index...
index created!
loading annotations into memory...
Done (t=0.62s)
creating index...
index created!
json_category_id_to_contiguous_id =  {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 27: 25, 28: 26, 31: 27, 32: 28, 33: 29, 34: 30, 35: 31, 36: 32, 37: 33, 38: 34, 39: 35, 40: 36, 41: 37, 42: 38, 43: 39, 44: 40, 46: 41, 47: 42, 48: 43, 49: 44, 50: 45, 51: 46, 52: 47, 53: 48, 54: 49, 55: 50, 56: 51, 57: 52, 58: 53, 59: 54, 60: 55, 61: 56, 62: 57, 63: 58, 64: 59, 65: 60, 67: 61, 70: 62, 72: 63, 73: 64, 74:

D:\softwareinstall\deeplearn\anaconda3\envs\paddle\lib\site-packages\paddle\fluid\executor.py:790: UserWarning: The current program is empty.
  warnings.warn(error_info)
Traceback (most recent call last):
  File "astar2019-evauate/score.py", line 187, in <module>
    score, mAP, flops = compute_score(args.model_dir, args.data_dir, batch_size=args.batch_size)
  File "astar2019-evauate/score.py", line 161, in compute_score
    feeded_var_names, feeder, target_var, batch_size)
  File "astar2019-evauate/score.py", line 86, in use_coco_api_compute_mAP
    cocoDt = cocoGt.loadRes(tmp_file)
  File "D:\softwareinstall\deeplearn\anaconda3\envs\paddle\lib\site-packages\pycocotools\coco.py", line 316, in loadRes
    anns = json.load(open(resFile))
  File "D:\softwareinstall\deeplearn\anaconda3\envs\paddle\lib\json\__init__.py", line 293, in load
    return loads(fp.read(),
MemoryError
W0114 15:56:20.981066  4908 device_context.cc:235] Please NOTE: device: 0, CUDA Capability: 61, Driver API Versio

**量化训练**

此部分用的是main_quant.py里的量化方法

也可以用PaddleSlim里的方法，详见work/paddleslim，同时PaddleSlim也可以进行剪枝等操作

In [ ]:
!python trian/main_quant.py  --data_dir F:\imagesource\coco\coco2017 --init_model model/model_snet/best_model --model_save_dir model/model_snet/snet_int8 --mode train

执行量化后的评测

In [6]:
!python astar2019-evauate/score.py --batch_size 32 --data_dir F:\imagesource\coco\coco2017 --model_dir model/model_snet/snet_int8/300_300_int8_inference

-----------  Configuration Arguments -----------
ap_version: cocoMAP
batch_size: 16
data_dir: dataset/coco2017
mean_value_B: 127.5
mean_value_G: 127.5
mean_value_R: 127.5
model_dir: model/model_snet/snet_int8/300_300_int8_inference
nms_threshold: 0.45
test_list: 
------------------------------------------------
height =  300
width =  300
loading annotations into memory...
Done (t=0.61s)
creating index...
index created!
loading annotations into memory...
Done (t=0.75s)
creating index...
index created!
json_category_id_to_contiguous_id =  {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 27: 25, 28: 26, 31: 27, 32: 28, 33: 29, 34: 30, 35: 31, 36: 32, 37: 33, 38: 34, 39: 35, 40: 36, 41: 37, 42: 38, 43: 39, 44: 40, 46: 41, 47: 42, 48: 43, 49: 44, 50: 45, 51: 46, 52: 47, 53: 48, 54: 49, 55: 50, 56: 51, 57: 52, 58: 53, 59: 54, 60: 55, 61: 56, 62: 57, 63: 58, 64: 59, 65

D:\softwareinstall\deeplearn\anaconda3\envs\paddle\lib\site-packages\paddle\fluid\executor.py:790: UserWarning: The current program is empty.
  warnings.warn(error_info)
D:\softwareinstall\deeplearn\anaconda3\envs\paddle\lib\site-packages\paddle\fluid\executor.py:774: UserWarning: The following exception is not an EOF exception.
  "The following exception is not an EOF exception.")
Traceback (most recent call last):
  File "astar2019-evauate/score.py", line 187, in <module>
    score, mAP, flops = compute_score(args.model_dir, args.data_dir, batch_size=args.batch_size)
  File "astar2019-evauate/score.py", line 161, in compute_score
    feeded_var_names, feeder, target_var, batch_size)
  File "astar2019-evauate/score.py", line 70, in use_coco_api_compute_mAP
    fetch_list=target_var)
  File "D:\softwareinstall\deeplearn\anaconda3\envs\paddle\lib\site-packages\paddle\fluid\executor.py", line 775, in run
    six.reraise(*sys.exc_info())
  File "D:\softwareinstall\deeplearn\anaconda3\envs

模型提交

In [2]:
#模型提交指令
!rm -rf submit.sh
!wget -O submit.sh http://ai-studio-static.bj.bcebos.com/script/submit.sh
!sh submit.sh model/model_snet/300_300_SSD.zip 18a12c8188884304b3f638f143c90047

'rm' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
'wget' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
'sh' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [4]:
#模型提交2
!cd model/model_snet/snet_int8/ && rm -rf submit.sh
!cd model/model_snet/snet_int8/ && wget -O submit.sh http://ai-studio-static.bj.bcebos.com/script/submit.sh
!cd model/model_snet/snet_int8/ && sh submit.sh 300_300_SSD.zip token

'rm' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
'wget' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
'sh' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
